In [1]:
import os
os.chdir('..')

from seg_game_typehinted import SevenSegButtonGame, MiniGame
from rpi_tm1638_animations import TM1638Animated as Tm

In [2]:
from random import randint
from typing import Any, Dict, List

In [3]:
math_answer_num = randint(1, 256)
math_answer_num_str = str(math_answer_num)
math_win_length = len(math_answer_num_str)


def math_setup(tm1638: Tm) -> Dict[str, Any]:
    """
    Setup answers and starting display for math game
    """
    # genrate the answer at random
    answer_list = [int(i) for i in math_answer_num_str]
    
    # assign random segment positions to the answer integers
    answer_int_positions = {}
    for i in range(len(answer_list)):
        rand_position = randint(0, tm1638.num_segments - 1)
        while rand_position in answer_int_positions:
            rand_position = randint(0, tm1638.num_segments - 1)
        answer_int_positions[rand_position] = int(answer_list[i])
    
    # generate answer sequence and starting display
    start_seg_display = [randint(0, 9) if i not in answer_int_positions
                         else answer_int_positions[i]
                         for i in range(tm1638.num_segments)]
        
    
    print('-----------------------------')
    print(math_answer_num)
    print(math_win_length)
    print(answer_int_positions)
    print('-----------------------------')
    print(answer_list)
    print(start_seg_display)
    print(f"{math_answer_num} ({format(math_answer_num, '08b')})")
        
    return_dict = {
        'correct_answer_conditions': answer_list,
        'game_seg_display': start_seg_display,
        'game_LED_display': math_answer_num
    }
        
    return return_dict


def math_map_input(input_button: int,
                   game_seg_display: List[int]) -> int:
    """
    Maps the button input to corrsonding the segment display input
    """
    input_button = game_seg_display[input_button]
    print(f'selected_segment_int: {input_button}')
    
    return input_button


def math_incorrect_answer_action(tm1638: Tm) -> List[Any]:
    """
    Reset progress if incorrect for math game
    """
    print("Error!")
    return 0


math_game = MiniGame(win_length=math_win_length,
                     setup_routine=math_setup,
                     map_input = math_map_input,
                     incorrect_answer_action=math_incorrect_answer_action,
                     show_button_feedback=False,
                     input_as_linear_int=True)

seg_game = SevenSegButtonGame(stb=2,
                              clk=3,
                              dio=4,
                              test_mode=True)
seg_game.register_game('math', math_game)
seg_game.setup()

-----------------------------
51
2
{3: 5, 5: 1}
-----------------------------
[5, 1]
[8, 0, 2, 5, 7, 1, 8, 1]
51 (00110011)


In [4]:
# Check math game answers (all power of 2)
math_game.correct_answer_conditions

[5, 1]

In [5]:
# Check the start display
math_game.game_seg_display

[8, 0, 2, 5, 7, 1, 8, 1]

In [6]:
def test_play(input):
    print(f'playing: {input}')
    math_game.play(1 << input)

In [7]:
test_play(0)

playing: 0
1 converted to: 0
selected_segment_int: 8
Error!
 •   •   *   *   •   •   *   *  
 _   _   _   _   _   _   _   _  
|_| |_| |_| |_| |_| |_| |_| |_| 
|_|.|_|.|_|.|_|.|_|.|_|.|_|.|_|.


In [8]:
# Check all answers correct wins game
for answer in math_game.correct_answer_conditions:
    answer_pos = math_game.game_seg_display.index(answer)
    test_play(answer_pos)

playing: 3
8 converted to: 3
selected_segment_int: 5
 •   •   *   *   •   •   *   *  
 _   _   _   _   _   _   _   _  
|_| |_| |_| |_| |_| |_| |_| |_| 
|_|.|_|.|_|.|_|.|_|.|_|.|_|.|_|.
playing: 5
32 converted to: 5
selected_segment_int: 1
 _   _   _   _   _   _   _   _  
|_| |_| |_| |_| |_| |_| |_| |_| 
|_|.|_|.|_|.|_|.|_|.|_|.|_|.|_|.


In [9]:
# Check the game stays finished
test_play(0)

playing: 0
 _   _   _   _   _   _   _   _  
|_| |_| |_| |_| |_| |_| |_| |_| 
|_|.|_|.|_|.|_|.|_|.|_|.|_|.|_|.
